Beat-n game with E10 base dishes and custom 5-pt jobs

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JuMP
using HiGHS

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [46]:
const ability_none = 999
const ability_add1 = 100
const ability_your_H_their_P = 400
const ability_your_F_their_L = 450
const ability_your_L_their_S = 500
const ability_your_P_their_L = 545
const ability_your_P_their_R = 550
const ability_your_R_their_S = 600

ability_name = Dict(
    ability_add1 => "add1",
    ability_your_H_their_P => "yourHtheirP",
    ability_your_F_their_L => "yourFtheirL",
    ability_your_L_their_S => "yourLtheirS",
    ability_your_P_their_R => "yourPtheirL",
    ability_your_P_their_R => "yourPtheirR",
    ability_your_R_their_S => "yourRtheirS",
)

function fx_add1(a::Spud, b::Spud)
    b0 = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.s, ability_none)
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+1, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+1, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p, a.r+1, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+1, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p+1, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p+1, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l+1, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l+1, a.p, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f+1, a.l, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f+1, a.l, a.p, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h+1, a.f, a.l, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h+1, a.f, a.l, a.p, a.r, a.s, ability_none)
    end
    return [a, b]
end

function fx_your_H_their_P(a::Spud, b::Spud)
    a = Spud(a.name, b.p, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, b.l, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, b.s, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, b.l, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, b.r, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, b.s, a.s, a.a1)
    return [a, b]
end



ability_fx = Dict(
    ability_add1 => fx_add1,
    ability_your_H_their_P => fx_your_H_their_P,
    ability_your_F_their_L => fx_your_F_their_L,
    ability_your_L_their_S => fx_your_L_their_S,
    ability_your_P_their_L => fx_your_P_their_L,
    ability_your_P_their_R => fx_your_P_their_R,
    ability_your_R_their_S => fx_your_R_their_S,
)

Dict{Int64, Function} with 7 entries:
  450 => fx_your_F_their_L
  545 => fx_your_P_their_L
  500 => fx_your_L_their_S
  550 => fx_your_P_their_R
  600 => fx_your_R_their_S
  100 => fx_add1
  400 => fx_your_H_their_P

In [5]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [6]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [7]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [8]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [9]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [10]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get dishes with cost 10

In [13]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff_i += 1
                                ff = Spud(string("d_", h,f,l,p,r,s), h,f,l,p,r,s,999)
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [14]:
dishes = get_library(10);
#dishes

In [15]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [16]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [17]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

#i2s = Dict(spud2int(ff) => ff for ff in lib)

spud2int (generic function with 1 method)

## Custom jobs

In [18]:
function combine_jobs_dishes(jobs::Array{Spud}, dishes::Array{Spud})::Array{Spud}
    lib = Array{Spud}(undef, 10000)
    ff_i = 0
    for j in jobs
        for d in dishes
            ff = Spud(string(j.name, " ", d.name), j.h+d.h, j.f+d.f, j.l+d.l, j.p+d.p, j.r+d.r, j.s+d.s, j.a1)
            ff_i += 1
            lib[ff_i] = ff
        end
    end
    lib = lib[1:ff_i];
    return lib
end

function get_nash_env(jobs::Array{Spud}, dishes::Array{Spud})::Tuple{Array{Spud}, Array{Float64}}
    lib = combine_jobs_dishes(jobs, dishes)
    ## optimize
    payoffs = get_payoffs(lib);
    vector_model = Model(HiGHS.Optimizer)
    n_nash = length(lib)
    sumv = zeros(Int64, (1, 2 * n_nash + 1))
    sumv[1:n_nash] .= 1
    A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
    A = vcat(A, sumv);
    b = zeros(Int64, n_nash + 1);
    b[1:n_nash] .= -1
    b[n_nash+1] = 1
    transpose(b);
    c = zeros(Int64, 2 * n_nash + 1);
    c[2 * n_nash + 1] = 1;
    @variable(vector_model, x[1:(2*n_nash+1)] >= 0);
    @constraint(vector_model, A * x .== b);
    @objective(vector_model, Min, c' * x)
    @time optimize!(vector_model)
    ## solution
    w_lp = [value(x[i]) for i in 1:n_nash]
    w_lp = w_lp./sum(w_lp);
    new_env = lib[w_lp .> 0]
    w_new = w_lp[w_lp .> 0];
    for i in sortperm(-w_new)
        print(new_env[i])
        print(" ")
        println(string(w_new[i])[1:6])
    end
    return new_env, w_new
end

get_nash_env (generic function with 1 method)

In [37]:
jobs = [
    Spud("Ascetic",     2,1,0,0,1,1,ability_none),
    Spud("Dancer",      1,0,1,1,0,1,ability_none),
    Spud("Singer",      0,0,2,1,1,0,ability_none),
    #Spud("Empath",     0,3,2,1,0,0,ability_none),
    Spud("Brawler",     0,0,1,2,1,1,ability_none),
    #Spud("Merchant",   0,1,2,0,0,1,ability_none),
    Spud("Salesman",    0,2,1,1,1,0,ability_none),
    Spud("Driver",      1,1,0,0,1,2,ability_none),
    Spud("Pyro",        0,0,0,3,0,0,ability_your_R_their_S),
    Spud("Hustler",     0,0,0,0,1,3,ability_none),
    Spud("Janitor",     0,0,0,2,0,2,ability_none),
    Spud("Photographer",0,1,0,0,0,2,ability_your_P_their_L),
    Spud("Student",     0,1,0,1,0,0,ability_add1)
]
ne, wn = get_nash_env(jobs, dishes);

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1387 rows, 2773 cols, 1923318 nonzeros
1387 rows, 2599 cols, 1682350 nonzeros
Presolve : Reductions: rows 1387(-0); columns 2599(-174); elements 1682350(-240968)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1387(1387) 0s
       1703     1.0000000000e+00 Pr: 0(0) 0s
       1703     1.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1703
Objective value     :  1.0000000000e+00
HiGHS run time      :          0.95
  1.013088 seconds (22.41 k allocations: 112.403 MiB)
Spud("Singer d_412111", 4, 1, 4, 2, 2, 1, 999) 0.0587
Spud("Student d_121114", 1, 3, 1, 2, 1, 4, 100) 0.0568
Spud("Driver d_111142", 2, 2, 1, 1, 5, 4, 999) 0.0483
Spud("Singer d_313111", 3, 1, 5, 2, 2, 1, 999) 0.0449
Spud("Ascetic d_214111", 4

In [38]:
df = spuds_to_df(ne)
df[!, :w] = [floor(w*1000)/1000 for w in wn]
print(df[sortperm(-wn),:])

55×9 DataFrame
 Row │ name               h      f      l      p      r      s      a1     w       
     │ String             Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64 
─────┼─────────────────────────────────────────────────────────────────────────────
   1 │ Singer d_412111        4      1      4      2      2      1    999    0.058
   2 │ Student d_121114       1      3      1      2      1      4    100    0.056
   3 │ Driver d_111142        2      2      1      1      5      4    999    0.048
   4 │ Singer d_313111        3      1      5      2      2      1    999    0.044
   5 │ Ascetic d_214111       4      2      4      1      2      2    999    0.037
   6 │ Pyro d_131212          1      3      1      5      1      2    600    0.036
   7 │ Pyro d_141112          1      4      1      4      1      2    600    0.034
   8 │ Brawler d_113221       1      1      4      4      3      2    999    0.033
   9 │ Dancer d_214111        3      1      5      2      1      2   

In [39]:
lib = combine_jobs_dishes(jobs, dishes)
payoffs = get_payoffs(lib)
nlib = length(lib)
combos = Array{Spud}(undef, (10000,3))
ord = sortperm([rand() for i in 1:nlib])[1:500]
#ord = [i for i in 1:nlib]
nc = 0
# 3-combos
for i in ord[1:(length(ord)-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(length(ord)-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            #print(".")
            for k in ord[(j+1):length(ord)]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = lib[[i,j,k]]
                    #println("")
                    #println("----3-COMBO!-----")
                    #println(lib[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:];

451 488 122 195 608 665 1156 229 1064 13 781 226 504 182 442 415 752 396 79 506 734 947 909 915 984 230 563 1192 134 575 895 1139 233 496 90 590 276 991 522 704 769 689 203 640 701 271 955 492 1119 254 643 1199 694 1207 970 8 272 205 690 920 901 756 739 1050 318 1178 385 484 995 489 294 1093 257 674 1008 460 687 130 1203 950 539 611 747 811 371 679 1033 1061 526 511 463 436 810 1258 976 1237 37 1351 1299 58 177 1171 391 161 1288 247 166 151 1290 625 241 361 695 1159 1036 102 708 189 1270 737 635 929 5 996 992 799 1322 180 1279 435 347 1167 572 54 768 921 1197 743 1376 1300 1057 174 812 617 1253 1071 417 1011 420 527 525 1040 707 127 1174 907 455 633 364 798 1 1332 975 450 763 808 1132 368 1285 1356 340 1065 1126 610 616 71 961 1146 10 1369 890 713 823 928 423 206 663 141 67 268 856 362 749 239 650 158 299 50 963 623 848 159 399 147 753 1032 7 938 772 732 131 619 1173 404 267 1089 1354 444 948 512 1133 1137 641 1202 341 892 1004 372 307 968 87 729 692 1378 223 427 807 62 352 219 465 129

In [40]:
size(combos)

(1, 3)

### combo-breaker research

In [41]:
as = combos[1,:]
print(spuds_to_df(as))

3×8 DataFrame
 Row │ name               h      f      l      p      r      s      a1    
     │ String             Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼────────────────────────────────────────────────────────────────────
   1 │ Ascetic d_111232       3      2      1      2      4      3    999
   2 │ Dancer d_313111        4      1      4      2      1      2    999
   3 │ Salesman d_121312      1      4      2      4      2      2    999

In [49]:
println(maximum([eval_battle_list(ff, as) for ff in lib]))
b = Spud("breaker",1,2,4,1,1,4,ability_your_H_their_P)
println([eval_battle(b, ff) for ff in as])
eval_battle_list2(b, ne, wn)

1
[1, 1, 1]


-0.2275359693748003